<a href="https://colab.research.google.com/github/urosgodnov/BigData/blob/master/textMining_scraping_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Permanently install libraries

## Initializing gdrive

In [1]:
# permanently install packageg in google colab
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Creating links

In [5]:
import os
nb_path = '/content/drive/MyDrive/Google_Colab_modules'

if not os.path.exists(nb_path):
  os.symlink('/content/drive/MyDrive/Google_Colab_modules', nb_path)  # Create the symlink only if it doesn't exist
else:
  print(f"Path '{nb_path}' already exists. Skipping symlink creation.")


Path '/content/drive/MyDrive/Google_Colab_modules' already exists. Skipping symlink creation.


## Installing to destination folder in gdrive

In [ ]:
#install the module in the
!pip install --target=$nb_path selenium

In [ ]:
!pip install --target=$nb_path webdriver_manager

## Connecting the path

In the future, while all modules will be installed, we will only use this command.

In [25]:
import sys

sys.path.append("/content/drive/MyDrive/Google_Colab_modules")

# Webscraping

In [194]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    ElementClickInterceptedException,
    TimeoutException
)
from datetime import datetime



In [195]:
import pandas as pd
import time

In [196]:
options = webdriver.ChromeOptions()
options.add_argument("--verbose")
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument("--window-size=1920, 1200")
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like \
Gecko) Chrome/90.0.4430.212 Safari/537.36')

In [207]:
driver = webdriver.Chrome(options=options)

In [215]:
l="https://www.imdb.com/title/tt13622970/reviews/?ref_=tt_urv_sm"
driver.get(l)

In [216]:
wait = WebDriverWait(driver, timeout=35)


In [217]:
while True:
    try:
        # Wait for the "All" button to be clickable
        all_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'ipc-see-more__button') and .//span[text()='All']]")))
        print('Found the "All" button.')

        # Scroll the button into view
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", all_button)
        print('Scrolled to the "All" button.')

        # Click the "All" button
        all_button.click()
        print('Button clicked successfully!')

        # After clicking the "All" button, wait for the "25 more" button to appear
        wait.until(EC.presence_of_element_located((By.XPATH, "//span[text()='25 more']")))  # Wait for "25 more" button to appear
        print('Found the "25 more" button.')

    except ElementClickInterceptedException:
        print('Element click intercepted, retrying...')

        # Wait for the page to load or for the scroll bar to move
        while driver.execute_script("return document.readyState") != "complete":
            print('Waiting for page to load...')
            time.sleep(1)  # Sleep for 1 second before checking again
        print('Page is loaded.')

    except TimeoutException:
        print('Timeout waiting for the button to be clickable or for the "25 more" button to appear.')
        break

    except NoSuchElementException:
        print('Button not found on the page.')
        break

    except Exception as e:
        print('An unexpected error occurred:', e)
        break


Found the "All" button.
Scrolled to the "All" button.
Element click intercepted, retrying...
Page is loaded.
Found the "All" button.
Scrolled to the "All" button.
Element click intercepted, retrying...
Page is loaded.
Found the "All" button.
Scrolled to the "All" button.
Element click intercepted, retrying...
Page is loaded.
Found the "All" button.
Scrolled to the "All" button.
Element click intercepted, retrying...
Page is loaded.
Found the "All" button.
Scrolled to the "All" button.
Element click intercepted, retrying...
Page is loaded.
Found the "All" button.
Scrolled to the "All" button.
Element click intercepted, retrying...
Page is loaded.
Found the "All" button.
Scrolled to the "All" button.
Element click intercepted, retrying...
Page is loaded.
Found the "All" button.
Scrolled to the "All" button.
Element click intercepted, retrying...
Page is loaded.
Found the "All" button.
Scrolled to the "All" button.
Element click intercepted, retrying...
Page is loaded.
Found the "All" but

KeyboardInterrupt: 

In [212]:
from bs4 import BeautifulSoup
content = driver.page_source
soup = BeautifulSoup(content)
result=[]
df = pd.DataFrame(columns=["Author","Title","Rating","Text","Date"])

In [189]:
def GetAReview(review):
  author = rev.findAll('a', attrs={'class':'ipc-link ipc-link--base'})[0].text
  title = rev.find('h3', attrs={'class':'ipc-title__text'}).text.strip()
  rating =rev.findAll('span', attrs={'class':'ipc-rating-star--rating'})[0].text.replace("\n","").strip()
  review = rev.findAll('div', attrs={'class':'ipc-html-content-inner-div'})[0].text
  date = rev.findAll('li', attrs={'class':'ipc-inline-list__item review-date'})[0].text
  date=datetime.strptime(date, '%b %d, %Y').strftime('%Y-%m-%d')

  return pd.DataFrame({"Author":author, "Title":title, "Rating":rating, "Text":review, "Date":date},index=[0])


In [213]:
for rev in soup.findAll('article'):
  try:
    result.append(GetAReview(rev))
  except:
    continue

In [214]:
df=pd.concat(result, axis=0, ignore_index=True)
df.head()

,Author,Title,Rating,Text,Date
0,downthepond,Tragic Wasted Potential,6,Let me start with the positives. There are a L...,2024-10-03
1,julie-ford,Future cult classic,9,Every scene of this film wowed me at TIFF. The...,2024-09-07
2,stevenmonkeydishwasher,Was great but lost it at the end,7,Was an enjoyable watch but the last 20-30 minu...,2024-10-14
3,FeastMode,My single most memorable movie experience,10,And I've seen thousands. I estimate about 5k m...,2024-09-19
4,heisenberg12,"Don't Buy the Hype, I Want My Money and Time Back",3,This is a desperately overrated film. It's an ...,2024-09-24


In [ ]:
df.to_csv("Elevation.csv")